<a href="https://colab.research.google.com/github/ShavaRobert/GISTaskSheets/blob/master/EO_in_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Earth Engine (GEE) 

GEE is a cloud-based platform for massive-scale analysis of geospatial data. GEE allows users access to a petabyte-scale archive of publicly available remotely sensed imagery, ancillary data and computational tools to accomplish a myriad of remote sensing and geospatial tasks at unprecedented speeds and scales. GEE is free for non-commercial use provided users sign up for a GEE account.

## Earth Engine (EE) Code Editor

Earth Engine is available through Python and JavaScript Application Program Interfaces (APIs). The JavaScript API is accessible via a webbased Integrated Development Environment (IDE) called the Code Editor. Through  this platform users can write and execute scripts to share and repeat geospatial analysis and processing workflows. To access Code Editor one can navigate to the URL [https://code.earthengine.google.com/](https://code.earthengine.google.com/) on Google Chrome web browser. 

The Python API alternative to Earth Engine Javascript API is Google Colaboratory.

## Google Colaboratory 'Colab'

Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser, with Zero configuration required, Free access to GPUs, and Easy sharing.

This [Welcome to Colaboratory](https://colab.research.google.com/notebooks/intro.ipynb?utm_source=scs-index#scrollTo=GJBs_flRovLc) document has all the necessary info to get started with Colab.


### Accounts

#### Google:

To get started with Colab, you will need a [Google Account](https://accounts.google.com/signup/v2/webcreateaccount?flowName=GlifWebSignIn&flowEntry=SignUp). This can be @gmail or an institution/custom google account. You will use this account to access Google Drive, Google Colab, and activate Google Earth Engine.

To use the ONS google account, you will require a Google Cloud Titan security key to access the EE code editor. One can obtain this key through ONS Service Desk. 


#### Google Earth Engine:

You wil also need an account for Google Earth Engine. You can [sign up here](https://accounts.google.com/ServiceLogin/webreauth?service=ah&passive=true&continue=https%3A%2F%2Fuc.appengine.google.com%2F_ah%2Fconflogin%3Fcontinue%3Dhttps%3A%2F%2Fsignup.earthengine.google.com%2F&flowName=GlifWebSignIn&flowEntry=ServiceLogin), using your Google Account from the previous step to link everything together nicely. 


### Connections

#### Connect Google Drive:

You can access files in Drive in a number of ways, including:

- Mounting your Google Drive in the runtime's virtual machine
- Using a wrapper around the API such as [PyDrive](https://pythonhosted.org/PyDrive/)
- Using the [native REST API](https://developers.google.com/drive/api/v3/about-sdk)

#### Mounting Google Drive locally

The example below shows how to mount your Google Drive on your runtime using an authorization code. 
Run the code block below. The output will show a link you have to open, copy the code from the page that loads and paste back into a cell provided

In [1]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


**Explore your drive**    

Now that you've connected (mounted) google drive to your session, you can access it as a local disk.

Look at the panel on the left, for the folder icon (3rd from the top). You should see your google drive under drive where you should see My Drive and Shared Drives. At any point if you hover over a folder three vertical dots appear on the right, and if you click you will see the option to copy path.

Explore using python to list files:

In [2]:
import os

# list the folders and files in your drive
mydrive = '/content/drive/My Drive'
colab = '/content/drive/My Drive/Colab Notebooks'
print(os.listdir(mydrive))

# list the current working directory
print(os.getcwd())

# # create a sub-folder called data under your 'Colab Notebooks'
data_folder = os.path.join(colab,'data')
os.path.isdir(data_folder)
if (not os.path.isdir(data_folder)):
  os.mkdir(data_folder)
# list the contents
print(os.listdir(colab))

# root directory for outputs is set to your google drive
my_root_dir = "/content/drive/My Drive/Colab Notebooks/data"

['How to get started with Drive.pdf', 'Colab Notebooks']
/content
['data', 'Copy of EO_in_Google_Colab.ipynb', 'EO_in_Google_Colab.ipynb']


Connect to Google Earth Engine and Initialise

Next you will need to install the Google Earth Engine python API. If running on Colab it's already installed. It will autodetect and skip if already installed.

In [3]:
# If not on Colab you'll need install the earth-engine Python API
#!pip install earthengine-api #earth-engine Python API

# Athenticate to your GEE account. 
!earthengine authenticate

# Initialize Earth Engine Python API
import ee
ee.Initialize()

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=ToALi2MVtKXzFGZUyCOsLC__qStg6YmwfBWjdmXTi04&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhnBXXZhlMW3PyZHr1i3uuL2ANuPe5yC8XxOVHmKkXFlWl4yot8Y60

Successfully saved authorization token.


Do a Search

Query Earth Engine (data collections) by looking for imagery of Scotland

Install packages if required

In [4]:
# Requirements, will skip if already installed
%pip install geopandas rasterio rasterstats shapely
%pip install folium earthengine-api

     |████████████████████████████████| 994 kB 7.4 MB/s 
     |████████████████████████████████| 19.3 MB 7.1 MB/s 
     |████████████████████████████████| 15.4 MB 38 kB/s 
     |████████████████████████████████| 6.6 MB 46.8 MB/s 
     |████████████████████████████████| 129 kB 60.9 MB/s 


**Import libraries**

In [5]:
# import libraries that you require for your analysis
from os import path as op
import pickle

import geopandas as gpd
import shapely as shp
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio.features import rasterize
from rasterstats.io import bounds_window
import rasterstats
import folium

### Uploading files from your local file system

files.upload returns a dictionary of the files which were uploaded. The dictionary is keyed by the file name and values are the data which were uploaded.

## Do a search
Let's query Earth Engine [data collections] by looking for Imagery of UK.

We are going to:

Define a bounding box or center point  using [geojson.io](https://geojson.io/#map=6/54.489/-3.922)     
Identify the collection   
Query and create an ImageCollection   
Display on a map   
Export   
Reload Raster and Explore   

In [6]:
# Define  bounding box using 

# Make sure we have the libraries we need
# %pip install folium
# %pip install geopandas



In [7]:
# Paste the Geojson from geojsonio
aoi_geojson = '''{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -0.59326171875,
              61.1167129205518
            ],
            [
              -4.37255859375,
              59.31076795603884
            ],
            [
              -7.9541015625,
              58.35563036280964
            ],
            [
              -8.876953125,
              55.25407706707272
            ],
            [
              -6.240234374999999,
              49.781264058178344
            ],
            [
              -3.779296875,
              50.00773901463687
            ],
            [
              -3.2958984375,
              50.41551870402678
            ],
            [
              -1.2084960937499998,
              50.45750402042058
            ],
            [
              1.47216796875,
              50.90303283111257
            ],
            [
              2.04345703125,
              52.77618568896171
            ],
            [
              0.59326171875,
              53.63161060657857
            ],
            [
              -1.29638671875,
              55.640398956687356
            ],
            [
              -1.12060546875,
              56.897003921272606
            ],
            [
              -0.59326171875,
              61.1167129205518
            ]
          ]
        ]
      }
    }
  ]
}'''

# Now let's read the geojson into a GeoPandasDataframe
aoi = gpd.read_file(aoi_geojson)
print(aoi) # so we can see what it looks like

#Get the bounding box
bbox = aoi.total_bounds
print(bbox) # see the coordinates

#Make it a GEE rectangle
gee_aoi = ee.Geometry.Rectangle(bbox.tolist())
center = aoi.centroid[0]

                                            geometry
0  POLYGON ((-0.59326 61.11671, -4.37256 59.31077...
[-8.87695312 49.78126406  2.04345703 61.11671292]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:85: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [8]:
# load map

m = folium.Map(location=[center.y, center.x], tiles="OpenStreetMap", zoom_start=12)

folium.Marker(
    location=[center.y, center.x],
    popup='Center',
    icon=folium.Icon(color='green', icon='ok-sign'),
).add_to(m)

folium.features.GeoJson(aoi_geojson,
                        style_function = lambda x: {'color':'blue', 'fillcolor':'transparent'}
                        ).add_to(m)
    
m

## Query Google Earth Engine
Now lets pick a collection of imagery to query for the region. We can choose from anything in the Google Earth Engine [data catalog](https://developers.google.com/earth-engine/datasets/catalog)

Let's start with the [Landsat 8 Surface Reflectance](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2) from 2020, using a cloud mask and selecting the bands to make an R,G,B mosiac

In [9]:
# To make a map we first need some helper functions

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [10]:
# Query GEE for Landsat

l8_image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterDate('2020-01-01', '2020-08-14')\
    .median()

# Time to make a map
l8_vis_params = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
}

Mapdisplay(center=[center.y, center.x],
           dicc={'L8':l8_image.getMapId(l8_vis_params)}, 
           zoom_start=12)

## Sub-setting and exporting to google Drive
At some point we often want to export data from Earth Engine for other uses. You may export because you need access to algorithms, data, or map making tools that just aren't possible in Earth Engine.


In [11]:
# Subset to our aoi
band_sel = ('B2', 'B3', 'B4', 'B5')

l8_image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterDate('2020-01-01', '2020-08-31')\
    .select(band_sel)\
    .filterBounds(gee_aoi)\
    .median()

l8_image.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'double',
    'type': 'PixelType'},
   'id': 'B2'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'double',
    'type': 'PixelType'},
   'id': 'B3'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'double',
    'type': 'PixelType'},
   'id': 'B4'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'double',
    'type': 'PixelType'},
   'id': 'B5'}],
 'type': 'Image'}

In [12]:
# Only run this when necessary, once exported you can use the file from google drive.
# Landsat 8 in this example takes about 10 minutes or less.
task.start()

NameError: ignored